# Create annotations from predictions

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from object_detection.utils import label_map_util, visualization_utils, config_util
from object_detection.builders import model_builder

%matplotlib inline

In [3]:
paths ={
    'pipeline_config': os.path.join('ASL_AI', 'models', 'ASL_prototype_labeling', 'pipeline.config'),
    'checkpoint_path': os.path.join('ASL_AI', 'models', 'ASL_prototype_labeling'),
    'label_map': os.path.join('ASL_AI', 'annotations', 'ASL_label_map.pbtxt'),
    'images_path': os.path.join('Images', 'Captured_ASL', 'to_be_labeled')
}

if not os.path.exists(paths['images_path']):
    if os.name == 'posix':
        !mkdir -p {paths['images_path']}
    if os.name == 'nt':
        !mkdir {paths['images_path']}

image_files = os.listdir(paths['images_path'])
category_index = label_map_util.create_category_index_from_labelmap(paths['label_map']) 
configs = config_util.get_configs_from_pipeline_file(paths['pipeline_config'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['checkpoint_path'], 'ckpt-3')).expect_partial()

@tf.function
def detect_object(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## Function for creating Annotations

<annotation>
	<folder>rockerhorns</folder>
	<filename>rockerhorns0a6752a3-91fb-11ed-b261-ec2e98cc4096.jpg</filename>
	<path>C:\Users\redth\Desktop\Object Detection Tensorflow\tensorflow\workspace\images\colledtedimages\rockerhorns\rockerhorns0a6752a3-91fb-11ed-b261-ec2e98cc4096.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>640</width>
		<height>480</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>RockerHorns</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>197</xmin>
			<ymin>151</ymin>
			<xmax>333</xmax>
			<ymax>354</ymax>
		</bndbox>
	</object>
</annotation>

In [4]:
from string import ascii_uppercase
label_map = {}
counter = 1
for char in ascii_uppercase:
    label_map[counter] = 'ASL_'+char
    counter += 1
print(label_map)


def pred_to_annotation(img_path, image_np, detections, classes):
    filename = img_path.split('\\')[3]
    path = img_path
    width = image_np.shape[1]
    height = image_np.shape[0]
    depth = 3
    object_name = label_map[classes + 1]
    x_min=int(detections[0][1] * width)
    y_min=int(detections[0][0] * height)
    x_max=int(detections[0][3] * width)
    y_max=int(detections[0][2] * height)
    annotation = f"""<annotation>
    \t<folder>{object_name}</folder>
    \t<filename>{filename}</filename>
    \t<path>{path}</path>
    \t<source>
    \t\t<database>Unknown</database>
    \t</source>
    \t<size>
    \t\t<width>{width}</width>
    \t\t<height>{height}</height>
    \t\t<depth>{depth}</depth>
    \t</size>
    \t<segmented>0</segmented>
    \t<object>
    \t\t<name>{object_name}</name>
    \t\t<pose>Unspecified</pose>
    \t\t<truncated>0</truncated>
    \t\t<difficult>0</difficult>
    \t\t<bndbox>
    \t\t\t<xmin>{x_min}</xmin>
    \t\t\t<ymin>{y_min}</ymin>
    \t\t\t<xmax>{x_max}</xmax>
    \t\t\t<ymax>{y_max}</ymax>
    \t\t</bndbox>
    \t</object>
    </annotation>
    """
    
    annotation_path = img_path.replace('.jpg', '.xml')
    with open(annotation_path, 'w') as f:
        f.write(annotation)

{1: 'ASL_A', 2: 'ASL_B', 3: 'ASL_C', 4: 'ASL_D', 5: 'ASL_E', 6: 'ASL_F', 7: 'ASL_G', 8: 'ASL_H', 9: 'ASL_I', 10: 'ASL_J', 11: 'ASL_K', 12: 'ASL_L', 13: 'ASL_M', 14: 'ASL_N', 15: 'ASL_O', 16: 'ASL_P', 17: 'ASL_Q', 18: 'ASL_R', 19: 'ASL_S', 20: 'ASL_T', 21: 'ASL_U', 22: 'ASL_V', 23: 'ASL_W', 24: 'ASL_X', 25: 'ASL_Y', 26: 'ASL_Z'}


In [5]:
img_paths = [os.path.join(paths['images_path'],file) for file in image_files if file.endswith('.jpg')]
for img_path in img_paths:

    img = cv2.imread(img_path)
    image_np = np.array(img)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_object(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    #print(detections['detection_boxes'][0])
    
    #visualization_utils.visualize_boxes_and_labels_on_image_array(
    #    image_np_with_detections,
    #    detections['detection_boxes'],
    #    detections['detection_classes'] + label_id_offset,
    #    detections['detection_scores'],
    #    category_index,
    #    use_normalized_coordinates=True,
    #    max_boxes_to_draw=5,
    #    min_score_thresh=.8,
    #    agnostic_mode=False
    #)
    
    #plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    #plt.show()
    pred_to_annotation(img_path, image_np, detections['detection_boxes'], detections['detection_classes'][0])